<a href="https://colab.research.google.com/github/jhmendonca/stablediffusion/blob/main/Diffusers_TREINAMENTO_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Os modelos de difusão são uma técnica de geração de imagem que usa uma rede neural para aprender como remover o ruído de uma imagem. A rede neural é treinada em um conjunto de dados de imagens e ruído. Conforme a rede neural é treinada, ela aprende a identificar e remover o ruído das imagens.

A ideia por trás dos modelos de difusores é adicionar ruído a uma imagem e em seguida, usar uma rede neural para remover o ruído gradualmente. Conforme o ruído é removido, a imagem começa a se transformar na imagem original.

Os modelos de difusão estável são um tipo especial de modelo de difusão chamado modelo de difusão latente .

 O modelo de Difusão original tende a consumir muito mais memória, então foram criados modelos de difusão latentes que podem fazer o processo de difusão em um espaço de dimensão inferior chamado Espaço Latente.

 Em difusão latente, o modelo é treinado para fazer esse mesmo processo em uma dimensão inferior.

Existem três componentes principais na difusão latente -

1. Um codificador de texto, neste caso, um codificador de texto CLIP

2. Um autoencoder, neste caso, um Variational Auto Encoder também   conhecido como VAE

3. Uma Convolutional Networks U-NET

O codificador de texto CLIP (Contrastive Language–Image Pre-training) usa o texto como uma entrada e gera incorporações de texto próximas.

![](https://huggingface.co/jhmendonca/stablediffusion-webui/resolve/main/0001.png)

Qualquer modelo de aprendizado de máquina não entende dados de texto. Para que qualquer modelo entenda os dados de texto, precisamos converter esse texto em números que contenham o significado do texto, referido como embeddings. O processo de conversão de um texto em um número pode ser dividido em duas partes -
1. Tokenizer - Quebrar cada palavra em subpalavras e em seguida, usar uma tabela de pesquisa para convertê-las em um número
2. Codificador Token_To_Embedding - Converter esses sub numéricos de palavras em uma representação que contém a representação desse texto

Vamos ver através do código. Começaremos importando os artefatos relevantes.

In [ ]:
!pip install -qq -U transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.1 MB/s eta 0:00:00


In [ ]:
import torch, logging
## disable warnings
logging.disable(logging.WARNING)
## Import the CLIP artifacts
from transformers import CLIPTextModel, CLIPTokenizer
## Initiating tokenizer and encoder.
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16)
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16).to("cuda")

Vamos inicializar um prompt e tokenizá-lo.

In [ ]:
prompt = [ "a man with glasses" ]
tok =tokenizer(prompt, padding= "max_length" , max_length=tokenizer.model_max_length, truncation= True , return_tensors= "pt" )
print (tok.input_ids.shape)
tok

torch.Size([1, 77])


{'input_ids': tensor([[49406,   320,   786,   593,  6116, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]])}

O tokenizador retorna dois objetos na forma de um dicionário.
1. input_ids: Um tensor de tamanho 1x77 como um prompt foi passado e preenchido para 77 comprimento máximo, 49406 é um token inicial, 320 é um token dado à palavra **“a”**, 768 à palavra "**man**", 593 à palavra "**with**", 3801 à palavra "**glasses**", e 49407 é o token de fim de texto repetido até o comprimento do bloco de 77.
2. attention_mask: 1 representando um valor incorporado e 0 representando preenchimento.

In [ ]:
for token in list(tok.input_ids[0,:7]):
    print(f"{token}:{tokenizer.convert_ids_to_tokens(int(token))}")

49406:<|startoftext|>
320:a</w>
786:man</w>
593:with</w>
6116:glasses</w>
49407:<|endoftext|>
49407:<|endoftext|>


Então, vamos ver o Token_To_Embedding Encoder que pega o input_ids gerado pelo tokenizador e os converte em incorporações -

In [ ]:
emb = text_encoder(tok.input_ids.to("cuda"))[0].half()
print(f"Shape of embedding : {emb.shape}")
emb

Shape of embedding : torch.Size([1, 77, 768])


tensor([[[-0.3884,  0.0229, -0.0523,  ..., -0.4902, -0.3066,  0.0674],
         [ 0.0292, -1.3242,  0.3076,  ..., -0.5254,  0.9766,  0.6655],
         [ 0.9937,  0.2395,  0.0254,  ..., -1.6729,  0.3599,  0.5112],
         ...,
         [-0.3435,  1.1621,  1.6270,  ..., -1.9541, -2.2949,  1.0986],
         [-0.3333,  1.1631,  1.6572,  ..., -1.9697, -2.2832,  1.1025],
         [-0.3662,  1.1572,  1.6416,  ..., -1.9199, -2.1973,  1.0674]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<NativeLayerNormBackward0>)

Como podemos ver acima, cada entrada tokenizada de tamanho 1x77 agora foi traduzida para incorporação de forma 1x77x768. Assim, cada palavra foi representada em um espaço de 768 dimensões.

![](https://huggingface.co/jhmendonca/stablediffusion-webui/resolve/main/vae.png)

In [ ]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")



In [ ]:
url = "https://s2-autoesporte.glbimg.com/aDGEGDKKz7LSZL7AsDI2EykDwmQ=/0x0:620x413/1000x0/smart/filters:strip_icc()/i.s3.glbimg.com/v1/AUTH_cf9d035bf26b4646b105bd958f32089d/internal_photos/bs/2020/a/4/Ik8J1fQYirf6wYRvRJ8Q/2020-03-20-novo-tracker-1.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a bike", "a photo of a car"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities
print(probs)

tensor([[1.6621e-05, 9.9998e-01]], grad_fn=<SoftmaxBackward0>)
